In [34]:
import os
import re
import glob
import shutil
import json
import random
import numpy as np
import nibabel as nib
import torchio as tio


/home/orco/anaconda3/envs/nnunet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classes = {1 : 'eye',
          2 : 'nerve',
          3: 'tumor',
          4: 'ret_det'}

# Grab SIDs

In [5]:
masks = glob.glob('/home/orco/data/Orbita_AI/Dataset/labels/*/*.nii.gz')
sids = [i.split('/')[-2] for i in masks]
len(sids)

30

# Make nnUNet Dataset

In [12]:
dataset_name = 'Dataset001_test'
task_n = re.match(r'Dataset(\d{3})_.*', dataset_name)[1]
print(task_n, dataset_name)

001 Dataset001_test


In [26]:
n_train = 30
train_sids = random.sample(sids, n_train)
test_sids = [i for i in sids if i not in train_sids]

## imgs and labels paths

In [69]:
imgsm = '../../Dataset/images/{sid}.nii.gz'
lblsm = '../../Dataset/labels/{sid}/{sid}.nii.gz'

In [70]:
dest_imgsm = '../../nnUNet/nnUNet_raw/{ds}/imagesTr/OrbitAI_{sid}_0000.nii.gz'
dest_lblsm = '../../nnUNet/nnUNet_raw/{ds}/labelsTr/OrbitAI_{sid}.nii.gz'

In [71]:
def recv_and(l):
    if len(l) > 1:
        return l[0] and recv_and(l[1:])
    else:
        return l[0]

In [72]:
recv_and([os.path.isfile(imgsm.format(sid=i)) for i in sids])

True

In [73]:
recv_and([os.path.isfile(lblsm.format(sid=i)) for i in sids])

True

## Copy files and set labels to UINT8

In [74]:
for i in train_sids:
    src_img = imgsm.format(sid=i)
    dest_img = dest_imgsm.format(ds=dataset_name, sid = i)
    if not os.path.isdir(os.path.dirname(dest_img)):
        os.makedirs(os.path.dirname(dest_img))
    shutil.copyfile(src_img, dest_img)
    src_lbl = lblsm.format(sid=i)
    dest_lbl = dest_lblsm.format(ds=dataset_name, sid = i)
    if not os.path.isdir(os.path.dirname(dest_lbl)):
        os.makedirs(os.path.dirname(dest_lbl))
    nii = nib.load(src_lbl)
    nii_arr = nii.get_fdata().astype(np.uint8)
    nnii = nib.Nifti1Image(nii_arr, header=nii.header, affine=nii.affine)
    nib.save(nnii, dest_lbl)

## Augmentation

In [75]:
for n,i in enumerate(train_sids):
    src_img = dest_imgsm.format(ds=dataset_name, sid = i)
    src_lbl = dest_lblsm.format(ds=dataset_name, sid = i)
    tr1 = tio.RandomFlip(axes=('LR'))
    tr2 = tio.RandomAffine(scales=(0.95,1.05),
                           degrees=5)
    tr = tio.Compose([tr1, tr2])
    subject = tio.Subject(
        img = tio.ScalarImage(src_img),
        lbl = tio.LabelMap(src_lbl))
    dest_img = dest_imgsm.format(ds=dataset_name, sid = str(n).zfill(8))
    dest_lbl = dest_lblsm.format(ds=dataset_name, sid = str(n).zfill(8))
    tr_sub = tr(subject)
    tr_sub.img.save(dest_img)
    tr_sub.lbl.save(dest_lbl)

## Make dataset.json

In [76]:
ds_json = '../../nnUNet/nnUNet_raw/{ds}/dataset.json'.format(ds=dataset_name)

In [81]:
s = '''{ 
 "name": "OrbitAI", 
 "description": "OrbitAI",
 "tensorImageSize": "4D",
 "channel_names": { 
   "0": "T2"
 }, 
 "labels": { 
   "background": 0, 
   "eye": 1,
   "nerve": 2,
   "tumor": 3,
   "ret_det": 4
 }, 
 "numTraining": %s, 
 "file_ending": ".nii.gz"
 }''' % (len(train_sids)*2)
#

In [82]:
s

'{ \n "name": "OrbitAI", \n "description": "OrbitAI",\n "tensorImageSize": "4D",\n "channel_names": { \n   "0": "T2"\n }, \n "labels": { \n   "background": 0, \n   "eye": 1,\n   "nerve": 2,\n   "tumor": 3,\n   "ret_det": 4\n }, \n "numTraining": 60, \n "file_ending": ".nii.gz"\n }'

In [83]:
j = json.loads(s)

In [84]:
with open(ds_json, 'w') as f:
    json.dump(j, f)